# Clinical Data Pipeline
Sebastian Quirarte Justo | Nov 2025 | sebastianquirajus@outlook.com

This notebook demonstrates an end-to-end clinical data engineering pipeline using Python and Pandas.  
It simulates the process used in regulated clinical environments (CDISC/SDTM) to transform raw operational files into validated, analysis-ready datasets.

Simulated SDTM formatted data was obtained from https://cdiscdataset.com/. 

Key concepts demonstrated:
- Loading raw clinical datasets
- Initial structural and quality checks
- Cleaning and standardizing data
- Joining clinical domains
- Creating analysis-ready outputs

This notebook is the development version of the pipeline, which will later be refactored into `.py` modeules.

### Contents

1. [Imports](#1-imports)
2. [Load SDTM Datasets](#2-load-sdtm-datasets)
3. [Exploratory Analysis and QC](#3-exploratory-analysis-and-quality-control)
4. [Data Cleaning & Standardization](#3-data-cleaning-&-Standardization)
5. Joining Data
6. Analysis & Outputs
7. Next Steps for Pipeline Deployment

### 1. Imports

In [ ]:
import pandas as pd # data manipulation
import numpy as np # numerical operations
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization

pd.set_option('display.max_columns', None)

### 2. Load SDTM Datasets

The raw SDTM CSV files are stored in the `/data` directory.

Each dataset is read into a pandas DataFrame:

- **DM:** Demographics  
- **AE:** Adverse Events  
- **EX:** Exposure  
- **VS:** Vital Signs 

In [ ]:
dm = pd.read_csv("data/DM.csv")
ae = pd.read_csv("data/AE.csv")
ex = pd.read_csv("data/EX.csv")
vs = pd.read_csv("data/VS.csv")

### 3. Exploratory Analysis and Quality Control
- Check for missing key variables  
- Confirm expected variables, such as `STUDYID` and `USUBJID`

In [ ]:
# Number of rows and columns and view first records of each dataset
  
datasets = {"DM": dm, "AE": ae, "EX": ex,"VS": vs}

for name, df in datasets.items():
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    display(df.head())


=== DM ===
Shape: (100, 25)
Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'SUBJID', 'RFSTDTC', 'RFENDTC', 'RFXSTDTC', 'RFXENDTC', 'RFICDTC', 'RFPENDTC', 'DTHDTC', 'DTHFL', 'SITEID', 'AGE', 'AGEU', 'SEX', 'RACE', 'ETHNIC', 'ARMCD', 'ARM', 'ACTARMCD', 'ACTARM', 'COUNTRY', 'DMDTC', 'DMDY']


,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,DTHDTC,DTHFL,SITEID,AGE,AGEU,SEX,RACE,ETHNIC,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY
0,STUDY001,DM,STUDY001-SUBJ0001,SUBJ0001,2025-06-28,2025-11-02,2025-07-04,2025-11-02,2024-12-16,2025-11-02,NaN,NaN,16,39,YEARS,M,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,ARM2,Treatment 2,ARM2,Treatment 2,ST,2025-07-13,17
1,STUDY001,DM,STUDY001-SUBJ0002,SUBJ0002,2025-09-21,2025-11-03,2025-07-31,2025-10-27,2025-07-22,2025-11-03,NaN,NaN,2,21,YEARS,M,ASIAN,HISPANIC OR LATINO,ARM2,Treatment 2,ARM2,Treatment 2,MK,2025-03-15,17
2,STUDY001,DM,STUDY001-SUBJ0003,SUBJ0003,2025-03-13,2025-10-31,2025-05-21,2025-10-30,2025-01-27,2025-10-31,NaN,NaN,32,48,YEARS,F,AMERICAN INDIAN OR ALASKA NATIVE,HISPANIC OR LATINO,ARM1,Treatment 1,ARM1,Treatment 1,SD,2025-10-22,5
3,STUDY001,DM,STUDY001-SUBJ0004,SUBJ0004,2025-09-30,2025-11-18,2025-05-06,2025-11-14,2025-05-28,2025-11-18,NaN,NaN,19,24,YEARS,F,AMERICAN INDIAN OR ALASKA NATIVE,NOT HISPANIC OR LATINO,ARM3,Treatment 3,ARM3,Treatment 3,UY,2024-12-08,19
4,STUDY001,DM,STUDY001-SUBJ0005,SUBJ0005,2024-12-21,2025-10-31,2025-10-13,2025-11-18,2025-05-08,2025-10-31,NaN,NaN,43,49,YEARS,F,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,ARM2,Treatment 2,ARM2,Treatment 2,TV,2025-05-19,24



=== AE ===
Shape: (285, 27)
Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'AESEQ', 'AESPID', 'AETERM', 'AEDECOD', 'AEBODSYS', 'AESOC', 'AESEV', 'AETOXGR', 'AESER', 'AEREL', 'AEACN', 'AEOUT', 'AESTDTC', 'AEENDTC', 'AESTDY', 'AEENDY', 'AEDUR', 'AECONTRT', 'AESDTH', 'AESLIFE', 'AESHOSP', 'AESDISAB', 'AESMIE', 'AEACNOTH']


,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AEDECOD,AEBODSYS,AESOC,AESEV,AETOXGR,AESER,AEREL,AEACN,AEOUT,AESTDTC,AEENDTC,AESTDY,AEENDY,AEDUR,AECONTRT,AESDTH,AESLIFE,AESHOSP,AESDISAB,AESMIE,AEACNOTH
0,STUDY001,AE,STUDY001-SUBJ0001,1,AE1,HEADACHE,HEADACHE,CARDIAC DISORDERS,CARDIAC DISORDERS,SEVERE,3,Y,NOT RELATED,DOSE NOT CHANGED,RECOVERED/RESOLVED,2025-10-02,NaN,70,NaN,NaN,N,N,N,N,N,N,NaN
1,STUDY001,AE,STUDY001-SUBJ0001,2,AE2,HEADACHE,HEADACHE,RESPIRATORY DISORDERS,RESPIRATORY DISORDERS,MILD,1,N,NOT RELATED,DOSE REDUCED,RECOVERING/RESOLVING,2025-09-09,NaN,47,NaN,NaN,Y,N,Y,N,N,Y,NaN
2,STUDY001,AE,STUDY001-SUBJ0001,3,AE3,RASH,RASH,NERVOUS SYSTEM DISORDERS,NERVOUS SYSTEM DISORDERS,MILD,1,N,RELATED,DRUG INTERRUPTED,NOT RECOVERED/NOT RESOLVED,2025-08-22,NaN,29,NaN,NaN,N,N,N,Y,N,N,NaN
3,STUDY001,AE,STUDY001-SUBJ0001,4,AE4,VOMITING,VOMITING,RESPIRATORY DISORDERS,RESPIRATORY DISORDERS,MODERATE,2,N,NOT RELATED,NOT APPLICABLE,RECOVERING/RESOLVING,2025-09-21,2025-11-18,59,117.0,58.0,N,N,Y,Y,N,Y,NaN
4,STUDY001,AE,STUDY001-SUBJ0001,5,AE5,FATIGUE,FATIGUE,CARDIAC DISORDERS,CARDIAC DISORDERS,MODERATE,2,N,NOT RELATED,DOSE NOT CHANGED,NOT RECOVERED/NOT RESOLVED,2025-09-28,2025-10-06,66,74.0,8.0,N,N,Y,Y,N,Y,NaN



=== EX ===
Shape: (547, 15)
Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'EXSEQ', 'EXTRT', 'EXDOSE', 'EXDOSU', 'EXDOSFRM', 'EXROUTE', 'EXDOSFRQ', 'EXSTDTC', 'EXENDTC', 'VISITNUM', 'VISIT', 'EXDY']


,STUDYID,DOMAIN,USUBJID,EXSEQ,EXTRT,EXDOSE,EXDOSU,EXDOSFRM,EXROUTE,EXDOSFRQ,EXSTDTC,EXENDTC,VISITNUM,VISIT,EXDY
0,STUDY001,EX,STUDY001-SUBJ0001,1,STUDY DRUG,100,mg,TABLET,ORAL,QD,2025-09-28,2025-09-28,1,BASELINE,1
1,STUDY001,EX,STUDY001-SUBJ0001,2,STUDY DRUG,100,mg,TABLET,ORAL,QD,2025-10-04,2025-10-04,2,WEEK 1,7
2,STUDY001,EX,STUDY001-SUBJ0001,3,STUDY DRUG,100,mg,TABLET,ORAL,QD,2025-10-11,2025-10-11,3,WEEK 2,14
3,STUDY001,EX,STUDY001-SUBJ0001,4,STUDY DRUG,100,mg,TABLET,ORAL,QD,2025-10-25,2025-10-25,4,WEEK 4,28
4,STUDY001,EX,STUDY001-SUBJ0002,1,STUDY DRUG,100,mg,TABLET,ORAL,QD,2024-11-29,2024-11-29,1,BASELINE,1



=== VS ===
Shape: (4500, 16)
Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'VSSEQ', 'VSTESTCD', 'VSTEST', 'VSORRES', 'VSORRESU', 'VSSTRESC', 'VSSTRESN', 'VSSTRESU', 'VSBLFL', 'VISIT', 'VISITNUM', 'VSDTC', 'VSDY']


,STUDYID,DOMAIN,USUBJID,VSSEQ,VSTESTCD,VSTEST,VSORRES,VSORRESU,VSSTRESC,VSSTRESN,VSSTRESU,VSBLFL,VISIT,VISITNUM,VSDTC,VSDY
0,STUDY001,VS,STUDY001-SUBJ0001,1,HEIGHT,Height,174.8,cm,174.8,174.8,cm,NaN,SCREENING,1,2025-03-05,-14
1,STUDY001,VS,STUDY001-SUBJ0001,2,WEIGHT,Weight,114.9,kg,114.9,114.9,kg,NaN,SCREENING,1,2025-03-05,-14
2,STUDY001,VS,STUDY001-SUBJ0001,3,TEMP,Temperature,37.0,C,37.0,37.0,C,NaN,SCREENING,1,2025-03-05,-14
3,STUDY001,VS,STUDY001-SUBJ0001,4,BP,Blood Pressure,142/98,mmHg,142/98,NaN,mmHg,NaN,SCREENING,1,2025-03-05,-14
4,STUDY001,VS,STUDY001-SUBJ0001,5,HR,Heart Rate,90,beats/min,90,90.0,beats/min,NaN,SCREENING,1,2025-03-05,-14


In [ ]:
# Check if participant ids are consistent across datasets

dm_subjects = set(dm["USUBJID"]) # unique participants ids in DM dataset

for name, df in datasets.items():
    if name != "DM":
        domain_subjects = set(df["USUBJID"])
        missing = dm_subjects - domain_subjects
        print(f"{name}: Subjects missing from domain = {len(missing)}")

AE: Subjects missing from domain = 16
EX: Subjects missing from domain = 0
VS: Subjects missing from domain = 0


It makes sense that not all participants will be in the AE dataset since we don't usually expect them all to present an AE.

In [ ]:
# Check that required columns are present and consistent across datasets

required_columns = {"DM": ["STUDYID", "USUBJID", "SEX", "ARM"],
                    "AE": ["STUDYID", "USUBJID", "AESTDTC", "AEDECOD"],
                    "EX": ["STUDYID", "USUBJID", "EXTRT", "EXSTDTC"],
                    "VS": ["STUDYID", "USUBJID", "VSTESTCD", "VSSTRESN"]}

for name, df in datasets.items():
    print(f"\n{name} required columns:")
    for col in required_columns[name]:
        if col in df.columns:
            print(f"  ✓ {col}")
        else:
            print(f"  ✗ Missing: {col}")


DM required columns:
  ✓ STUDYID
  ✓ USUBJID
  ✓ SEX
  ✓ ARM

AE required columns:
  ✓ STUDYID
  ✓ USUBJID
  ✓ AESTDTC
  ✓ AEDECOD

EX required columns:
  ✓ STUDYID
  ✓ USUBJID
  ✓ EXTRT
  ✓ EXSTDTC

VS required columns:
  ✓ STUDYID
  ✓ USUBJID
  ✓ VSTESTCD
  ✓ VSSTRESN


In [23]:
# Missing values across domains
for name, df in datasets.items():
    print(f"\n=== Missing Values in {name} ===")
    display(df.isna().sum())


=== Missing Values in DM ===


STUDYID       0
DOMAIN        0
USUBJID       0
SUBJID        0
RFSTDTC       0
RFENDTC       0
RFXSTDTC      0
RFXENDTC      0
RFICDTC       0
RFPENDTC      0
DTHDTC      100
DTHFL       100
SITEID        0
AGE           0
AGEU          0
SEX           0
RACE          0
ETHNIC        0
ARMCD         0
ARM           0
ACTARMCD      0
ACTARM        0
COUNTRY       1
DMDTC         0
DMDY          0
dtype: int64


=== Missing Values in AE ===


STUDYID       0
DOMAIN        0
USUBJID       0
AESEQ         0
AESPID        0
AETERM        0
AEDECOD       0
AEBODSYS      0
AESOC         0
AESEV         0
AETOXGR       0
AESER         0
AEREL         0
AEACN         0
AEOUT         0
AESTDTC       0
AEENDTC      80
AESTDY        0
AEENDY       80
AEDUR        80
AECONTRT      0
AESDTH        0
AESLIFE       0
AESHOSP       0
AESDISAB      0
AESMIE        0
AEACNOTH    285
dtype: int64


=== Missing Values in EX ===


STUDYID     0
DOMAIN      0
USUBJID     0
EXSEQ       0
EXTRT       0
EXDOSE      0
EXDOSU      0
EXDOSFRM    0
EXROUTE     0
EXDOSFRQ    0
EXSTDTC     0
EXENDTC     0
VISITNUM    0
VISIT       0
EXDY        0
dtype: int64


=== Missing Values in VS ===


STUDYID        0
DOMAIN         0
USUBJID        0
VSSEQ          0
VSTESTCD       0
VSTEST         0
VSORRES        0
VSORRESU       0
VSSTRESC       0
VSSTRESN     900
VSSTRESU       0
VSBLFL      4000
VISIT          0
VISITNUM       0
VSDTC          0
VSDY           0
dtype: int64

### 5. Data Cleaning & Standardization

In this section, we perform basic cleaning and standardization to prepare the datasets for analysis.

In [26]:
dm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   STUDYID   100 non-null    object 
 1   DOMAIN    100 non-null    object 
 2   USUBJID   100 non-null    object 
 3   SUBJID    100 non-null    object 
 4   RFSTDTC   100 non-null    object 
 5   RFENDTC   100 non-null    object 
 6   RFXSTDTC  100 non-null    object 
 7   RFXENDTC  100 non-null    object 
 8   RFICDTC   100 non-null    object 
 9   RFPENDTC  100 non-null    object 
 10  DTHDTC    0 non-null      float64
 11  DTHFL     0 non-null      float64
 12  SITEID    100 non-null    int64  
 13  AGE       100 non-null    int64  
 14  AGEU      100 non-null    object 
 15  SEX       100 non-null    object 
 16  RACE      100 non-null    object 
 17  ETHNIC    100 non-null    object 
 18  ARMCD     100 non-null    object 
 19  ARM       100 non-null    object 
 20  ACTARMCD  100 non-null    object 

In [30]:
# AE
ae['AESTDTC'] = pd.to_datetime(ae['AESTDTC'], errors='coerce')
ae['AEDECOD'] = ae['AEDECOD'].str.upper().str.strip()

# DM
dm['SEX'] = dm['SEX'].str.upper().map({'M':'M','F':'F'}).fillna('UNKNOWN')
dm['AGE'] = pd.to_numeric(dm['AGE'], errors='coerce')
dm['ARM'] = dm['ARM'].astype('category')

# EX
ex['EXSTDTC'] = pd.to_datetime(ex['EXSTDTC'], errors='coerce')
if 'EXENDTC' in ex.columns:
    ex['EXENDTC'] = pd.to_datetime(ex['EXENDTC'], errors='coerce')
if 'EXDOSE' in ex.columns:
    ex['EXDOSE'] = pd.to_numeric(ex['EXDOSE'], errors='coerce')
ex['EXTRT'] = ex['EXTRT'].str.upper().str.strip()

# VS
vs['VSDTC'] = pd.to_datetime(vs['VSDTC'], errors='coerce')
vs['VSSTRESN'] = pd.to_numeric(vs['VSSTRESN'], errors='coerce')
vs['VSTESTCD'] = vs['VSTESTCD'].str.upper().str.strip()
